# 📓 Guía paso a paso: Consumir API → SQLite (sqlite3) → Pandas → Gráficas con **Plotly**

En esta guía replicamos tu flujo con `sqlite3` y reemplazamos las gráficas de `matplotlib/seaborn` por **Plotly** para obtener visualizaciones **interactivas**.

**Fuente de datos:** `https://jsonplaceholder.typicode.com/users`

---
## Requisitos
Ejecuta:

```bash
pip install requests pandas plotly
```


In [1]:
pip install requests pandas plotly


Note: you may need to restart the kernel to use updated packages.


In [2]:
# 1) Importaciones
import requests
import sqlite3
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

DB_NAME = 'usuarios3.db'
API_URL = 'https://jsonplaceholder.typicode.com/users'


## 2) Consumir la API y validar respuesta

In [3]:
response = requests.get(API_URL, timeout=20)
if response.status_code != 200:
    raise SystemExit(f'❌ Error al consumir la API ({response.status_code})')
users = response.json()
print(f'Filas recibidas: {len(users)}')
users[:2]  # vista rápida


Filas recibidas: 10


[{'id': 1,
  'name': 'Leanne Graham',
  'username': 'Bret',
  'email': 'Sincere@april.biz',
  'address': {'street': 'Kulas Light',
   'suite': 'Apt. 556',
   'city': 'Gwenborough',
   'zipcode': '92998-3874',
   'geo': {'lat': '-37.3159', 'lng': '81.1496'}},
  'phone': '1-770-736-8031 x56442',
  'website': 'hildegard.org',
  'company': {'name': 'Romaguera-Crona',
   'catchPhrase': 'Multi-layered client-server neural-net',
   'bs': 'harness real-time e-markets'}},
 {'id': 2,
  'name': 'Ervin Howell',
  'username': 'Antonette',
  'email': 'Shanna@melissa.tv',
  'address': {'street': 'Victor Plains',
   'suite': 'Suite 879',
   'city': 'Wisokyburgh',
   'zipcode': '90566-7771',
   'geo': {'lat': '-43.9509', 'lng': '-34.4618'}},
  'phone': '010-692-6593 x09125',
  'website': 'anastasia.net',
  'company': {'name': 'Deckow-Crist',
   'catchPhrase': 'Proactive didactic contingency',
   'bs': 'synergize scalable supply-chains'}}]

## 3) Guardar en **SQLite** con `sqlite3` (tabla `users`)

In [4]:
conn = sqlite3.connect(DB_NAME)
cur = conn.cursor()

# (opcional) reiniciar la tabla para ser deterministas al re-ejecutar
cur.execute('DROP TABLE IF EXISTS users;')

cur.execute('''
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY,
    name TEXT,
    username TEXT,
    email TEXT,
    phone TEXT,
    website TEXT
)
''')

for u in users:
    cur.execute('''
        INSERT OR REPLACE INTO users (id, name, username, email, phone, website)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', (
        u.get('id'), u.get('name'), u.get('username'), u.get('email'), u.get('phone'), u.get('website')
    ))

conn.commit()
conn.close()
print('✅ Datos guardados exitosamente en SQLite')


✅ Datos guardados exitosamente en SQLite


## 4) Leer con **pandas**

In [5]:
conn = sqlite3.connect(DB_NAME)
df = pd.read_sql_query('SELECT * FROM users', conn)
conn.close()
df.head()


,id,name,username,email,phone,website
0,1,Leanne Graham,Bret,Sincere@april.biz,1-770-736-8031 x56442,hildegard.org
1,2,Ervin Howell,Antonette,Shanna@melissa.tv,010-692-6593 x09125,anastasia.net
2,3,Clementine Bauch,Samantha,Nathan@yesenia.net,1-463-123-4447,ramiro.info
3,4,Patricia Lebsack,Karianne,Julianne.OConner@kory.org,493-170-9623 x156,kale.biz
4,5,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,(254)954-1289,demarco.info


## 5) *Feature engineering* básico

In [6]:
# Longitud del nombre
df['name_length'] = df['name'].astype(str).apply(len)

# Dominio del email
df['email_domain'] = df['email'].astype(str).apply(lambda x: x.split('@')[-1].lower() if '@' in str(x) else None)

df[['id','name','name_length','email','email_domain']].head()


,id,name,name_length,email,email_domain
0,1,Leanne Graham,13,Sincere@april.biz,april.biz
1,2,Ervin Howell,12,Shanna@melissa.tv,melissa.tv
2,3,Clementine Bauch,16,Nathan@yesenia.net,yesenia.net
3,4,Patricia Lebsack,16,Julianne.OConner@kory.org,kory.org
4,5,Chelsey Dietrich,16,Lucio_Hettinger@annie.ca,annie.ca


## 6) Gráficas con **Plotly**

### 6.1 Histograma interactivo de `name_length`

In [7]:
fig = px.histogram(df, x='name_length', nbins=10, title='Distribución de caracteres en los nombres')
fig.update_layout(xaxis_title='Cantidad de caracteres', yaxis_title='Frecuencia')
fig.show()


### 6.2 Barras: usuarios por dominio de correo

In [8]:
dom_counts = df['email_domain'].value_counts().reset_index()
dom_counts.columns = ['email_domain','count']
fig = px.bar(dom_counts, x='count', y='email_domain', orientation='h',
             title='Usuarios por dominio de correo electrónico')
fig.update_layout(xaxis_title='Cantidad de usuarios', yaxis_title='Dominio')
fig.show()


### 6.3 (Opcional) Otras visualizaciones rápidas

In [9]:
# Pie / Donut de dominios
fig = px.pie(dom_counts, names='email_domain', values='count', hole=0.4,
             title='Distribución de dominios de email (Donut)')
fig.show()

# Tabla interactiva simple
fig = go.Figure(data=[go.Table(
    header=dict(values=list(df[['id','name','username','email','phone','website']].columns), align='left'),
    cells=dict(values=[df['id'], df['name'], df['username'], df['email'], df['phone'], df['website']], align='left')
)])
fig.update_layout(title='Usuarios (tabla)')
fig.show()


---
## 7) Exportar gráficos
Puedes exportar cualquier figura a HTML con `fig.write_html('archivo.html')`. Por ejemplo:

In [10]:
fig = px.histogram(df, x='name_length', nbins=10, title='Ejemplo de exportación')
fig.write_html('hist_name_length.html', include_plotlyjs='cdn')
print('Archivo exportado: hist_name_length.html')


Archivo exportado: hist_name_length.html
